# Word Embeddings
We will use word2vec with gensim to solve word similarity and analogy tasks. Use gensim's [word2vec tutorial](https://radimrehurek.com/gensim/auto_examples/tutorials/run_word2vec.html) for a reference of the functions we need.

First, we download the word embeddings pretrained on the (private) Google News corpus. The embeddings are quite big with 1.7 GB. They will be downloaded to your `<HOME>/gensim-data` directory.

In [ ]:
import gensim.downloader as api
wv = api.load('word2vec-google-news-300')

Let's look at the vector size, vocabulary size and its first 10 and last 5 words.

Get the vectors for woman, man, queen and king.

Compute their norms. Are they normalized?

## Word Similarity
We will now perform a few operations that use word similarity.

Given the pairs below...

In [ ]:
pairs = [
    ('car', 'minivan'),   # a minivan is a kind of car
    ('car', 'bicycle'),   # still a wheeled vehicle
    ('car', 'airplane'),  # ok, no wheels, but still a vehicle
    ('car', 'cereal'),    # ... and so on
    ('car', 'democracy'),
]

... compute the word similarity of each pair, once with [sklearn's cosine similarity](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html), and once with gensim's similarity.

Find the most similar word to all of the words: bike, train, plane, car, bus.

**Odd one out.** Find the term that doesn't fit the rest of the list: fire, water, land, car, sea, air.

Let's evaluate our word vectors on the WordSim353 dataset. You can get it from `datapath('wordsim353.tsv')` (import `datapath` from `gensim.test.utils`). What's the Pearson correlation?

## Word Analogy
We also want to evaluate our word embeddings on the word analogy task. The test set from the original word2vec paper also comes with gensim, and is found in `datapath('questions-words.txt')`. Let's look at the categories that are present in the test file.

In [ ]:
with open(datapath('questions-words.txt')) as f:
    for i, line in enumerate(f):
        if line.startswith(':'):
            print(i, line.strip())

To save us some computation time, we're just going to use the first category, `capital-common-countries`. We store its evaluation examples in a new text file `word-analogies.txt`.

In [ ]:
lines = []
with open(datapath('questions-words.txt')) as f:
    for line in f:
        if line.startswith(': capital-world'):
            break
        lines.append(line)
print(lines[:10])
with open(datapath('word-analogies.txt'), 'w') as f:
    f.writelines(lines)

Now we can evaluate our model with gensim's built-in function `evaluate_word_analogies`. Save the results to a variable `analogy_scores`.

Print the accuracy and show an example of a correctly solved analogy, and an incorrectly solved one.

## Visualization
Let's visualize our word embeddings in two dimensions. We use PCA to reduce the dimensionality of our 300-dim word vectors to 2. Below is a function `display_scatterplot` that takes a list of words as input and plots them in 2D.

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
%matplotlib inline

def display_scatterplot(model, words): # assumes all words are in the vocabulary
    word_vectors = [model[word] for word in words]
    twodim = PCA().fit_transform(word_vectors)[:,:2]
    plt.figure(figsize=(6,6))
    plt.scatter(twodim[:,0], twodim[:,1], edgecolors='k', c='r')
    for word, (x,y) in zip(words, twodim):
        plt.text(x + 0.03, y + 0.03, word)

Define a list of words that could be present in an analogy task. Plot their down-projected embeddings. Can you detect a vector for the relation of your analogy?

## Fun with words
Try your own skills in guessing the hidden word by entering words and receiving the cosine similarity on [Semantle](https://semantle.com/).